# vLLM

[vLLM](https://vllm.readthedocs.io/en/latest/index.html) is a fast and easy-to-use library for LLM inference and serving, offering:

* State-of-the-art serving throughput
* Efficient management of attention key and value memory with PagedAttention
* Continuous batching of incoming requests
* Optimized CUDA kernels

This notebooks goes over how to use a LLM with langchain and vLLM.

To use, you should have the `vllm` python package installed.

In [1]:
%pip install vllm langchain langchain-community transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 88.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 55.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 97.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 96.6

## Ngrok

In [2]:
%pip install pyngrok

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# Chỉ cần làm 1 lần
ngrok.set_auth_token(user_secrets.get_secret("NGROK_AUTH_TOKEN"))

In [4]:
# Mở cổng 8000
# public_url = ngrok.connect(8000)
tunnel = ngrok.connect(
    addr=8000,
    proto="http",
    domain="vast-alien-obviously.ngrok-free.app"
)
print("Ngrok URL:", tunnel.public_url)

Ngrok URL: https://vast-alien-obviously.ngrok-free.app


In [6]:
# ngrok.kill()  # Dừng tất cả các tunnel và ngrok process

## Serve a model

In [ ]:
!python3 -m vllm.entrypoints.openai.api_server \
  --model AITeamVN/GRPO-VI-Qwen2-7B-RAG \
  --tensor-parallel-size 2 \
  --host 0.0.0.0 --port 8000

INFO 06-07 07:07:20 [__init__.py:243] Automatically detected platform cuda.
2025-06-07 07:07:22.486251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749280042.700751     161 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749280042.769794     161 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO 06-07 07:07:38 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-07 07:07:38 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-07 07:07:38 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load

## Get response

In [ ]:
%pip install langchain_openai

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    temperature=0.3,
    top_p=0.9,
    # top_k=10,
    max_tokens=1000,
)
response = llm.invoke("Việt Nam có bao nhiêu tỉnh thành?")
print(response.content)